In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import glob

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EthicsDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        outputs = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }
        
        if self.labels is not None:
            outputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
            
        return outputs

In [29]:
def load_data(category, file_type):
    if category == 'utilitarianism':
        prefix = 'util'
    elif category == "commonsense":
        prefix = "cm"
    else:
        prefix = category
    
    file_path = os.path.join(category, f"{prefix}_{file_type}.csv")
    
    df = pd.read_csv(file_path)
    
    # Different categories of datasets have different structures, process them separately
    if category == 'deontology':
        texts = df['scenario'].astype(str) + ' ' + df['excuse'].astype(str)
        if 'label' in df.columns:
            labels = df['label'].values
        else:
            labels = None
    elif category == 'utilitarianism':
        columns = df.columns.tolist()
        texts = df[columns[0]].values
        # For data without labels, return None
        labels = None
    else:
        if 'label' in df.columns:
            labels = df['label'].values
            texts = df.iloc[:, 1:].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).values
        else:
            labels = None
            texts = df.iloc[:, 0:].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).values
    
    return texts, labels

In [25]:
def evaluate_model(model, dataloader):
    model.eval()
    
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
            
            _, preds = torch.max(outputs.logits, dim=1)
            
            predictions.extend(preds.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())
    
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
    
    return accuracy, precision, recall, f1

In [ ]:
def train_model(model, train_dataloader, val_dataloader, epochs=3):
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    
    # Total training steps
    total_steps = len(train_dataloader) * epochs
    
    # Create learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    
    # Training loop
    for epoch in range(epochs):
        print(f'Starting Epoch {epoch+1}/{epochs}')
        model.train()
        total_loss = 0
        
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)
            
            model.zero_grad()
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )
            
            loss = outputs.loss
            total_loss += loss.item()
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        
        avg_train_loss = total_loss / len(train_dataloader)
        print(f'Average training loss: {avg_train_loss:.4f}')
        
        # Evaluate model
        print('Evaluating on validation set...')
        accuracy, precision, recall, f1 = evaluate_model(model, val_dataloader)
        print(f'Validation accuracy: {accuracy:.4f}')
        print(f'Validation precision: {precision:.4f}')
        print(f'Validation recall: {recall:.4f}')
        print(f'Validation F1 score: {f1:.4f}')
        print()
    
    return model

In [27]:
def train_main():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    categories = [folder for folder in os.listdir() if os.path.isdir(folder) and folder != 'utilitarianism']
    categories.remove('.ipynb_checkpoints')
    print(f"Found the following categories: {categories}")
    
    all_train_texts = []
    all_train_labels = []
    
    for category in categories:
        print(f"Processing training data for category: {category}")
        
        train_texts, train_labels = load_data(category, 'train')
        all_train_texts.extend(train_texts)
        all_train_labels.extend(train_labels)
    
    train_dataset = EthicsDataset(all_train_texts, all_train_labels, tokenizer)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    
    val_texts, val_labels = load_data(categories[0], 'test')
    val_dataset = EthicsDataset(val_texts, val_labels, tokenizer)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=2,
        output_attentions=False,
        output_hidden_states=False,
    )
    
    model.to(device)
    
    model = train_model(model, train_dataloader, val_dataloader, epochs=3)
    model_save_path = 'ethics_model'
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print(f'Model saved to {model_save_path}')

In [30]:
train_main()

C:\Users\LYT\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Found the following categories: ['commonsense', 'deontology', 'justice', 'virtue']
Processing training data for category: commonsense
Processing training data for category: deontology
Processing training data for category: justice
Processing training data for category: virtue


C:\Users\LYT\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\LYT\anaconda3\Lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Starting Epoch 1/3
Average training loss: 0.3649
Evaluating on validation set...
Validation accuracy: 0.7743
Validation precision: 0.7789
Validation recall: 0.7782
Validation F1 score: 0.7742

Starting Epoch 2/3
Average training loss: 0.2203
Evaluating on validation set...
Validation accuracy: 0.8167
Validation precision: 0.8164
Validation recall: 0.8177
Validation F1 score: 0.8165

Starting Epoch 3/3
Average training loss: 0.1443
Evaluating on validation set...
Validation accuracy: 0.8386
Validation precision: 0.8379
Validation recall: 0.8380
Validation F1 score: 0.8380

Model saved to ethics_model


In [ ]:
tokenizer = BertTokenizer.from_pretrained('ethics_model')
print("===== Evaluating on each category separately =====")
model = BertForSequenceClassification.from_pretrained('ethics_model')
model.to(device)

categories = [folder for folder in os.listdir() if os.path.isdir(folder) and folder != 'utilitarianism']
categories.remove('.ipynb_checkpoints')
categories.remove('ethics_model')

for category in categories:
        print(f"\n----- Category: {category} -----")
        
        # Regular test set evaluation
        print(f'Evaluating final model on {category} regular test set...')
        test_texts, test_labels = load_data(category, 'test')
        test_dataset = EthicsDataset(test_texts, test_labels, tokenizer)
        test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(model, test_dataloader)
        print(f'{category} Test accuracy: {test_accuracy:.4f}')
        print(f'{category} Test precision: {test_precision:.4f}')
        print(f'{category} Test recall: {test_recall:.4f}')
        print(f'{category} Test F1 score: {test_f1:.4f}')
        
        print(f'\nEvaluating final model on {category} hard test set...')
        test_hard_texts, test_hard_labels = load_data(category, 'test_hard')
        test_hard_dataset = EthicsDataset(test_hard_texts, test_hard_labels, tokenizer)
        test_hard_dataloader = DataLoader(test_hard_dataset, batch_size=32, shuffle=False)
        
        test_hard_accuracy, test_hard_precision, test_hard_recall, test_hard_f1 = evaluate_model(model, test_hard_dataloader)
        print(f'{category} Hard test accuracy: {test_hard_accuracy:.4f}')
        print(f'{category} Hard test precision: {test_hard_precision:.4f}')
        print(f'{category} Hard test recall: {test_hard_recall:.4f}')
        print(f'{category} Hard test F1 score: {test_hard_f1:.4f}')

===== Evaluating on each category separately =====

----- Category: commonsense -----
Evaluating final model on commonsense regular test set...
commonsense Test accuracy: 0.8386
commonsense Test precision: 0.8379
commonsense Test recall: 0.8380
commonsense Test F1 score: 0.8380

Evaluating final model on commonsense hard test set...
commonsense Hard test accuracy: 0.4818
commonsense Hard test precision: 0.4843
commonsense Hard test recall: 0.4844
commonsense Hard test F1 score: 0.4816

----- Category: deontology -----
Evaluating final model on deontology regular test set...
deontology Test accuracy: 0.8142
deontology Test precision: 0.8168
deontology Test recall: 0.8145
deontology Test F1 score: 0.8140

Evaluating final model on deontology hard test set...
deontology Hard test accuracy: 0.6620
deontology Hard test precision: 0.6709
deontology Hard test recall: 0.6624
deontology Hard test F1 score: 0.6579

----- Category: justice -----
Evaluating final model on justice regular test set.